Убрать тексты на русском (но не на кабардинском/адыгейском) с помощью регулярок (лангдетект тут не поможет), при этом римские цифры оставляем

In [1]:
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 6.7 MB/s eta 0:00:00


In [5]:
import re
from wordfreq import zipf_frequency
import unicodedata
from collections import Counter

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
INPUT_FILE = "/content/drive/MyDrive/Adyghe_no_ocr_NFD.txt"
CLEAN_FILE = "/content/drive/MyDrive/Adyghe_clean_no_ru_long.txt"
REMOVED_FILE = "/content/drive/MyDrive/Adyghe_removed_ru_words.tsv"

with open(INPUT_FILE, encoding="utf-8") as f:
    text = f.read()

text = unicodedata.normalize("NFC", text)

STICKS = set("I")
HYPHENS = "-–—−"
ZIPF_THRESHOLD = 3.0

WORD_RE = re.compile(
    rf"[а-яёӀӏIiⅠⅼ|1{HYPHENS}]+",
    re.IGNORECASE
)

removed_counter = Counter()

# ====== ФУНКЦИЯ ДЛЯ re.sub ======
ZIPF_THRESHOLD = 3.0

def process_match(match):
    word = match.group(0)
    w = word.lower()

    # если есть палочка → NON-RU
    if any(ch in STICKS for ch in word):
        return word

    freq = zipf_frequency(w, "ru")

    # удаляем ТОЛЬКО если уверенно русское
    if freq >= ZIPF_THRESHOLD and len(w) > 3:
        removed_counter[w] += 1
        return ""
    else:
        return word

# ====== ОБРАБОТКА ТЕКСТА ======
clean_text = WORD_RE.sub(process_match, text)

# ====== СОХРАНЕНИЕ ======
with open(CLEAN_FILE, "w", encoding="utf-8") as f:
    f.write(clean_text)

with open(REMOVED_FILE, "w", encoding="utf-8") as f:
    f.write("word\tcount\tzipf_frequency\n")
    for w, cnt in removed_counter.most_common():
        freq = zipf_frequency(w, "ru")
        f.write(f"{w}\t{cnt}\t{freq:.2f}\n")

print("Очищенный текст сохранён:", CLEAN_FILE)
print("Удалённые слова сохранены:", REMOVED_FILE)
print("Уникальных удалённых слов:", len(removed_counter))
print("Всего удалённых вхождений:", sum(removed_counter.values()))


Очищенный текст сохранён: /content/drive/MyDrive/Adyghe_clean_no_ru_long.txt
Удалённые слова сохранены: /content/drive/MyDrive/Adyghe_removed_ru_words.tsv
Уникальных удалённых слов: 20244
Всего удалённых вхождений: 302621


Статистика

In [9]:
total_words = 0
ru_words = 0

for match in WORD_RE.finditer(text):
    word = match.group(0)
    w = word.lower()
    total_words += 1

    if any(ch in STICKS for ch in word):
        continue

    freq = zipf_frequency(w, "ru")

    if freq >= ZIPF_THRESHOLD and len(w) > 3:
        ru_words += 1

percent_ru = ru_words / total_words * 100 if total_words else 0

print("📊 СТАТИСТИКА")
print("Всего слов:", total_words)
print("Русских слов:", ru_words)
print(f"Процент русского текста: {percent_ru:.2f}%")

📊 СТАТИСТИКА
Всего слов: 11329083
Русских слов: 302621
Процент русского текста: 2.67%


Пример на фрагменте

In [23]:
text = """
«Зэкъошныгъ» - «Дагдизель»
04.09 - тхьаумаф
«Зэкъошныгъ» - «Кавказтрансгаз»
18.09 - тхьаумаф
«Зэкъошныгъ» - «Астрахань»
02.10 - тхьаумаф
«Зэкъошныгъ» - «Торпедо»
16.10 - тхьаумаф
«Зэкъошныгъ» - «Алания-Д»
30.10 - тхьаумаф
«Зэкъошныгъ» - «Динамо»
25.04 - бэрэскэжъый
«Зэкъошныгъ» - «Ангушт»
13.05 - тхьаумаф
«Зэкъошныгъ» - «Ротор»
"""


In [24]:
# ====== НАСТРОЙКИ ======
RU_SENT_THRESHOLD = 0.7
MIN_WORDS_IN_SENT = 3

STICKS = set("ӀӏIiⅠⅼ|1")
HYPHENS = "-–—−"

# ====== РЕГУЛЯРКИ ======
WORD_RE = re.compile(
    rf"(?:\p{{Cyrillic}}|\p{{Mn}}|\p{{Nd}}|[IiⅠⅼ|1]|[{HYPHENS}])+",
    re.IGNORECASE
)

SENT_RE = re.compile(r"[^.!?]+[.!?]?", re.DOTALL)

# ====== ФУНКЦИИ ======
def letter_len_nfd(word):
    return sum(
        1 for ch in word
        if unicodedata.category(ch).startswith("L")
    )

def has_stick(word):
    return any(ch in STICKS for ch in word)

def classify_word(word):
    """
    Возвращает:
    - "RU"
    - "NON-RU"
    - "SKIP"  (если длина <= 3)
    """
    if has_stick(word):
        return "NON-RU", 0.0

    length = letter_len_nfd(word)
    if length <= 3:
        return "SKIP", 0.0

    word_nfc = unicodedata.normalize("NFC", word)
    freq = zipf_frequency(word_nfc.lower(), "ru")

    if freq > 0:
        return "RU", freq
    return "NON-RU", 0.0

def is_ru_sentence(sent):
    words = WORD_RE.findall(sent)

    ru = 0
    considered = 0

    for w in words:
        label, _ = classify_word(w)

        if label == "SKIP":
            continue  # слова ≤3 букв не учитываем

        considered += 1
        if label == "RU":
            ru += 1

    if considered < MIN_WORDS_IN_SENT:
        return False

    return (ru / considered) >= RU_SENT_THRESHOLD

# ====== ОБРАБОТКА ======
print("=== КЛАССИФИКАЦИЯ СЛОВ ===\n")

for w in WORD_RE.findall(text):
    label, freq = classify_word(w)
    print(f"{w:<15} {label:<7} {freq:.2f}")

print("\n=== ПРОВЕРКА ПРЕДЛОЖЕНИЙ ===\n")

kept = []
removed = []

for sent in SENT_RE.findall(text):
    if is_ru_sentence(sent):
        removed.append(sent)
        print("[УДАЛЕНО]", sent.strip())
    else:
        kept.append(sent)
        print("[ОСТАЛОСЬ]", sent.strip())

print("\n=== ИТОГОВЫЙ ТЕКСТ ===\n")
print("".join(kept))


=== КЛАССИФИКАЦИЯ СЛОВ ===

Зэкъошныгъ      NON-RU  0.00
-               SKIP    0.00
Дагдизель       RU      1.63
04              SKIP    0.00
09              SKIP    0.00
-               SKIP    0.00
тхьаумаф        NON-RU  0.00
Зэкъошныгъ      NON-RU  0.00
-               SKIP    0.00
Кавказтрансгаз  NON-RU  0.00
18              NON-RU  0.00
09              SKIP    0.00
-               SKIP    0.00
тхьаумаф        NON-RU  0.00
Зэкъошныгъ      NON-RU  0.00
-               SKIP    0.00
Астрахань       RU      3.71
02              SKIP    0.00
10              NON-RU  0.00
-               SKIP    0.00
тхьаумаф        NON-RU  0.00
Зэкъошныгъ      NON-RU  0.00
-               SKIP    0.00
Торпедо         RU      4.12
16              NON-RU  0.00
10              NON-RU  0.00
-               SKIP    0.00
тхьаумаф        NON-RU  0.00
Зэкъошныгъ      NON-RU  0.00
-               SKIP    0.00
Алания-Д        RU      3.37
30              SKIP    0.00
10              NON-RU  0.00
-              